In [1]:
import cv2
import numpy as np
import os
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,Conv2D,Activation,Dropout
import tensorflow.keras.backend as K
import keras
from keras.models import Sequential, Model
from keras.models import load_model
from keras.optimizers import SGD
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras.layers import MaxPool2D
import tensorflow as tf
from tensorflow.python.keras.backend import get_session

In [2]:
train_path="C:/Users/admin/Desktop/New folder/train"
test_path="C:/Users/admin/Desktop/New folder/test"

In [3]:
image_dog=cv2.imread("C:/Users/admin/Desktop/New folder/train/cat.1.jpg")

In [4]:
from matplotlib import pyplot as plt

In [5]:
train_datagen = ImageDataGenerator(zoom_range=0.15,width_shift_range=0.2,height_shift_range=0.2,shear_range=0.15)
test_datagen = ImageDataGenerator()

In [6]:
train_generator = train_datagen.flow_from_directory("C:/Users/admin/Desktop/New folder/train",target_size=(224, 224),batch_size=32,shuffle=True)
test_generator = test_datagen.flow_from_directory("C:/Users/admin/Desktop/New folder/test",target_size=(224,224),batch_size=32,shuffle=False)

Found 25000 images belonging to 1 classes.
Found 12500 images belonging to 1 classes.


In [7]:
def VGG16():
    model = Sequential()
    model.add(Conv2D(input_shape=(224,224,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPool2D(pool_size=(2,2),strides=(2,2),name='vgg16'))
    model.add(Flatten(name='flatten'))
    model.add(Dense(256, activation='relu', name='fc1'))
    model.add(Dense(128, activation='relu', name='fc2'))
    model.add(Dense(1, activation='sigmoid', name='output'))
    return model

In [8]:
model=VGG16()

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 56, 56, 256)       2

In [10]:
Vgg16 = Model(inputs=model.input, outputs=model.get_layer('vgg16').output)

In [11]:
Vgg16.load_weights("C:/Users/admin/Desktop/New folder/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5")
for layer in Vgg16.layers:
    layer.trainable = False

In [12]:
for layer in model.layers:
    print(layer, layer.trainable)

<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BD36E20> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BDA3D90> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000028C9BDA3C10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BE61B50> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BE73370> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000028C9BE73C10> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BE7DB80> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BE84FD0> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BE917C0> False
<tensorflow.python.keras.layers.pooling.MaxPooling2D object at 0x0000028C9BE91310> False
<tensorflow.python.keras.layers.convolutional.Conv2D object at 0x0000028C9BE94520> False
<tensorflow.python.ke

In [13]:
opt = SGD(lr=1e-4, momentum=0.9)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

In [14]:
es=EarlyStopping(monitor='val_accuracy', mode='max', verbose=1, patience=20)

In [15]:
mc = ModelCheckpoint('C:/Users/admin/Desktop/New folder/best_model.h5', monitor='val_accuracy', mode='max', save_best_only=True)

In [16]:
model.load_weights("C:/Users/admin/Desktop/New folder/best_model.h5")

In [17]:
model_json = model.to_json()
with open("C:/Users/admin/Desktop/New folder/model.json","w") as json_file:
  json_file.write(model_json)

In [18]:
from keras.models import model_from_json

In [21]:
def predict_(image_path):
    #Load the Model from Json File
    json_file = open('C:/Users/admin/Desktop/New folder/model.json', 'r')
    model_json_c = json_file.read()
    json_file.close()
    model_c = model_from_json(model_json_c)
    #Load the weights
    model_c.load_weights("C:/Users/admin/Desktop/New folder/best_model.h5")
    #Compile the model
    opt = SGD(lr=1e-4, momentum=0.9)
    model_c.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
    #load the image you want to classify
    image = cv2.imread(image_path)
    image = cv2.resize(image, (224,224))
    from IPython.display import Image
    Image(image)
    #predict the image
    preds = model_c.predict(np.expand_dims(image, axis=0))[0]
    if preds==0:
        print("Predicted Label:Cat")
    else:
        print("Predicted Label: Dog")

In [23]:
predict_("C:/Users/admin/Desktop/ML/dog.jpg")

Predicted Label: Dog
